<a href="https://colab.research.google.com/github/SirTee12/LLM-Kaggle-Google-GenAI/blob/main/Prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install the SDK

In [ ]:
!pip uninstall -qqy jupyterlab # uninstall jupyterlab in the case it conflict google colab base image
!pip install -U -q "google-genai==1.7.0" # install google genai library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 3.3 MB/s eta 0:00:00


import the SDK and some helpers for rendering the output. The `types` in `import types` may include custom data structures, classes or type hints psecifically to work with google AI. These custom types might represent things like model parameters, input/output formats for AI models or config settings.

In [ ]:
from google import genai
from google.genai import types
from google.colab import userdata

import enum
import io
from IPython.display import HTML, Markdown, display, clear_output
import typing_extensions as typing

## API Call Retry Implementation
Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota


1.  **Imports `retry`:** It imports the `retry` module from `google.api_core`, which provides tools for automatic retries of API calls.

2.  **Defines `is_retriable`:**
    * It creates a lambda function called `is_retriable`.
    * This function checks if an exception (`e`) is an instance of `genai.errors.APIErrpr` and if its error code (`e.code`) is either 429 (Too Many Requests) or 503 (Service Unavailable).
    * It returns `True` if both conditions are met (meaning the API call is considered retriable), and `False` otherwise.

3.  **Applies Retry Logic:**
    * It modifies the `genai.models.Models.generate_content` function by wrapping it with the `retry.retry` function.
    * The `retry.retry` function is configured with the `is_retriable` function as the `predicate`. This tells `retry.retry` to only retry the API call if the `is_retriable` function returns `True` for the encountered exception.
    * Essentially, this replaces the original `generate_content` function with a new version that automatically retries on 429 and 503 errors, making the code more robust to temporary API issues.

In [ ]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in (429, 503))

genai.models.Models.generate_content = retry.Retry(predicate = is_retriable)(genai.models.Models.generate_content)

In [ ]:
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#secret_value_0 = user_secrets.get_secret("secret-label-prompt")

## Get the API keys and create a client

Retrieves the API key securely and create a client object that will be used to communicate with the generative AI service, using the retrieved API key for authentication.

In [ ]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY') # retrive the APi key
client = genai.Client(api_key=GOOGLE_API_KEY)   # create a client to interact with the genai application

## Running the First Prompt

In [ ]:
# generate a response
response = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = "Explain AI to me like I'm a kid"
)

print(response.text)
Markdown(response.text)

Okay, imagine you have a really smart puppy.  This puppy can learn tricks if you show it a lot, and a lot, and a lot of examples.  

AI is like that puppy, but instead of learning tricks, it learns to do other things.  

For example:

*   **Recognize pictures:** You can show it a million pictures of cats and dogs, and eventually, it will learn to tell the difference between them really well! It's like teaching the puppy "this is a cat!" and "this is a dog!" over and over again.
*   **Play games:** You can let it play a video game a million times, and it will learn how to play super well, maybe even better than you! It's like the puppy learning how to fetch the ball perfectly.
*   **Talk to you:** You can give it lots and lots of sentences, and it can learn to talk back to you, and even answer your questions.

The more examples you give the AI, the better it gets. It's not *really* thinking like you or me, but it's really good at finding patterns and doing what it's been trained to do.


Okay, imagine you have a really smart puppy.  This puppy can learn tricks if you show it a lot, and a lot, and a lot of examples.  

AI is like that puppy, but instead of learning tricks, it learns to do other things.  

For example:

*   **Recognize pictures:** You can show it a million pictures of cats and dogs, and eventually, it will learn to tell the difference between them really well! It's like teaching the puppy "this is a cat!" and "this is a dog!" over and over again.
*   **Play games:** You can let it play a video game a million times, and it will learn how to play super well, maybe even better than you! It's like the puppy learning how to fetch the ball perfectly.
*   **Talk to you:** You can give it lots and lots of sentences, and it can learn to talk back to you, and even answer your questions.

The more examples you give the AI, the better it gets. It's not *really* thinking like you or me, but it's really good at finding patterns and doing what it's been trained to do.

**Important:** AI is made by people. So, it's important to use it wisely and make sure it's fair to everyone!  Just like you need to train your puppy kindly, AI needs to be made and used with kindness too.

Does that make sense?


## Let's start a chat

In [ ]:
chat = client.chats.create(model = "gemini-2.0-flash", history = [])
response = chat.send_message('Hello, my name is Ahmad')
print(response.text)

Hello Ahmad, it's nice to meet you! How can I help you today?



In [ ]:
response = chat.send_message("can you tell me something interesting about dinosaurs")
print(response.text)

Okay, here's an interesting fact about dinosaurs:

**The Spinosaurus was likely the first known swimming dinosaur.**

While many dinosaurs were believed to live near water sources, the Spinosaurus had features that suggested it was actively swimming and hunting in rivers. These features include:

*   **Dense bones:** Unlike most dinosaurs which had hollow bones to reduce weight, Spinosaurus had dense bones similar to those of penguins, suggesting they were used as ballast for swimming.
*   **Paddle-like tail:** Its tail was much taller and broader than other dinosaurs, shaped like a paddle to propel it through the water.
*   **Nostrils positioned high on its skull:** This would allow it to breathe while partially submerged.
*   **Isotope analysis of teeth:** Showed that it ate both aquatic and terrestrial prey.

So, unlike the T-Rex which was a land-based predator, Spinosaurus seems to have adapted to a semi-aquatic lifestyle!



In [ ]:
response = chat.send_message("do you still remember my name")
print(response.text)

Yes, Ahmad! I remember your name.



# Chose a model from the Gemini model family

Retrieves a list of available AI models, searches for a specific model named "gemini-2.0-flash," and if found, displays its details in a formatted JSON-like structure. The search stops once the target model is located.

In [ ]:
for model in client.models.list():
  print(model)

name='models/chat-bison-001' display_name='PaLM 2 Chat (Legacy)' description='A legacy text-only model optimized for chat conversations' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=4096 output_token_limit=1024 supported_actions=['generateMessage', 'countMessageTokens']
name='models/text-bison-001' display_name='PaLM 2 (Legacy)' description='A legacy model that understands text and generates text as an output' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=8196 output_token_limit=1024 supported_actions=['generateText', 'countTextTokens', 'createTunedTextModel']
name='models/embedding-gecko-001' display_name='Embedding Gecko' description='Obtain a distributed representation of a text.' version='001' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, up

In [ ]:
from pprint import pprint

for model in client.models.list():
  if model.name == "models/gemini-2.0-flash":
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens', 'createCachedContent'],
 'tuned_model_info': {},
 'version': '2.0'}


## Parameters Exploration

### Output Length

In [ ]:
# set the max number of output token to 200
short_config = types.GenerateContentConfig(max_output_tokens = 200)

# create a response
response_short = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = short_config,
    contents='write a short essay on the importance of education in modern day society.'
)

print(response_short.text)

## The Bedrock of Progress: The Enduring Importance of Education in Modern Society

In the complex and rapidly evolving landscape of modern society, education stands as a cornerstone of progress, personal development, and societal advancement. More than simply acquiring knowledge, education empowers individuals, strengthens communities, and fuels innovation, making it an indispensable tool for navigating the challenges and opportunities of the 21st century.

Firstly, education is paramount for individual empowerment. It equips individuals with the critical thinking skills necessary to analyze information, solve problems, and make informed decisions. Literacy and numeracy, the fundamental building blocks of education, unlock access to information and participation in civic life. Beyond these basics, specialized education and training open doors to diverse career paths, providing individuals with the means to achieve economic independence and pursue their passions. Education fosters self

### Temperature

How much randomness is included when selecting the next word (token) in language models depends on the "temperature" parameter.  A higher temperature causes the model to take into account a greater number of potential words, producing more inventive and diverse results.  On the other hand, a lower temperature forces the model to adhere to the most likely terms, producing language that is more concentrated and predictable.  The model turns completely deterministic when the temperature is set to 0, always choosing the word that is the most likely.  Temperature does not, however, ensure actual randomness; rather, it is a means of directing the model toward more or less random results.

In [ ]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 2.0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Azure
 _________________________
Cerulean
 _________________________
Mauve
 _________________________
Turquoise
 _________________________
Magenta.
 _________________________


The previous output shows the randomness but we would experiment with temperature value of 0. we can see from the output that it is more direct and precise and there is no randomness.

In [ ]:
# create a variable to set the temperature
high_temp_config = types.GenerateContentConfig(temperature = 0)

for _ in range(5):
  response_temp = client.models.generate_content(
      model = 'gemini-2.0-flash',
      config = high_temp_config,
      contents = 'Pick a random colour... (respond in a single word)'
  )

  if response_temp.text:
    print(response_temp.text, '_' * 25)

Azure
 _________________________
Azure
 _________________________
Azure
 _________________________
Azure
 _________________________
Azure
 _________________________


### Top P

In [ ]:
model_config = types.GenerateContentConfig(
    temperature = 1.0, # default config
    top_p = 0.95       # default config
)

story_prompt = "You are a senior network engineer. Explain ospf to a 5 year old"
response_topp = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = model_config,
    contents=story_prompt
)

print(response_topp.text)

Okay, imagine we're playing a game with toy cars, and each car needs to deliver a package to different houses in our neighborhood.

You and your friends each have a toy car and a map of the neighborhood.  But everyone's map is a little different!

OSPF is like a special club we join so that EVERYONE has the *best* map of the neighborhood.

Here's how it works:

1. **Cars are like routers:** Each toy car is like a router. A router is like a tiny computer that helps messages (like your packages) travel from one place to another.

2. **Houses are like networks:** Each house is like a network – a group of computers or devices that are connected together.

3. **Roads are like connections:** The roads connecting the houses are like the connections between our routers.

4. **Sharing information:** OSPF helps all the toy cars (routers) talk to each other.  They say things like, "Hey, I know a shortcut to get to that house!  It's this way..." and "This road is closed because a giant teddy bear 

## Prompting

### Zero-shot Prompt

In [ ]:
# set the prompt parameters
model_config = types.GenerateContentConfig(
  temperature = 0.1,
  top_p = 1,
  max_output_tokens = 5
)

# set the prompt
zero_short_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response_zero_shot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = model_config,
    contents = zero_short_prompt
)

print(response_zero_shot.text) # print the output as text

POSITIVE



#### Enum Mode
The Sentiment enum acts as a schema or a constraint. It tells the language model that the expected output should be one of these three predefined sentiment categories.

The `text/x.enum` MIME type indicates that the response should be one of the values defined in the response_schema.

In [ ]:
# create a new enumeration class tha inherit from enum.Enum
class Sentiment(enum.Enum):
  POSITIVE = 'positive'
  NEUTRAL = 'neutral'
  NEGATIVE = 'negative'

response_zero_short_enum = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(

        # tell the model the ezpected output format and that the rsponse should
        # be one of thos defined in the schema
        response_mime_type = 'text/x.enum',

        # set the Sentiment as schema
        response_schema = Sentiment
    ),

    contents = zero_short_prompt
)
print(response_zero_shot.text)

POSITIVE



### Few Short Prompt

In [ ]:
few_shot_prompt = """Parse a customer's pizza order into valid JSON:

EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

ORDER:
"""

customer_order = 'Give me a large with cheese and pineapple'

response_one_few_shot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 1,
        max_output_tokens = 250
    ),
    contents = [few_shot_prompt, customer_order]
)

print(response_one_few_shot.text)

```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
```


#### JSON Mode

`class PizzaOrder(typing.TypedDict):`: This defines a new class called PizzaOrder that inherits from typing.TypedDict. TypedDict is used to create dictionary types where the keys and their corresponding value types are known

Purpose of the `TypedDict` is to act as a schema. It defines the structure and data types of the JSON output we expect from the language model. This allows us to work with the model's response in a structured and predictable way.


In [ ]:
class PizzaOrder(typing.TypedDict):
  size:str
  ingredients: list[str]
  type:str

response_json = client.models.generate_content(
    model = 'gemini-2.0-flash',
    config = types.GenerateContentConfig(
        temperature = 0.1,
        response_mime_type = 'application/json',
        response_schema = PizzaOrder
    ),

    contents = "Can I have a large dessert pizza with apple and chocolate"
)

print(response_json.text)

{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert"
}


## Chain of Thought

Chain of Thought prompting improves the language model's ability to perform complex reasoning tasks. By explicitly requesting the model to show its reasoning process, it's more likely to arrive at the correct answer, as it avoids impulsive or incorrect conclusions.

In [ ]:
# set up the prompt
cot_prompt = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? let's think step by step."""

response_cot = client.models.generate_content(
    model = 'gemini-2.0-flash',
    contents = cot_prompt)

Markdown(response_cot.text)


Here's how to solve the problem step-by-step:

1. **Find the age difference:** When you were 4, your partner was 3 times your age, meaning they were 4 * 3 = 12 years old.
2. **Calculate the age difference:** The age difference between you and your partner is 12 - 4 = 8 years.
3. **Determine the partner's current age:** Since the age difference remains constant, your partner is always 8 years older than you. Now that you are 20, your partner is 20 + 8 = 28 years old.

**Therefore, your partner is currently 28 years old.**


## Reason and Act: ReAct

In [ ]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# Come up with more examples yourself, or take a look through https://github.com/ysymyth/ReAct/

In [ ]:
# set the question
question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

# You will perform the Action; so generate up to, but not including, the Observation.
react_config = types.GenerateContentConfig(
    stop_sequences = ['\nObservation'],
    system_instruction = model_instructions + example1 + example2
    )

# Create a chat that has the model instructions and examples pre-seeded.
react_response = client.chats.create(
    model = 'gemini-2.0-flash',
    config = react_config
)

resp = react_response.send_message(question)

print(resp.text)

Thought 1
I need to find the transformers NLP paper, find the authors, and identify the youngest one.

Action 1
<search>transformers NLP paper</search>



# Thinking Mode


In [ ]:
# set a response
response_thinking = client.models.generate_content_stream(
    model = 'gemini-2.0-flash-thinking-exp',
    contents = 'Who was the youngest athor listed on the transofrmer NLP paper'
)

buffer = io.StringIO()
for chunk in response_thinking:
  buffer.write(chunk.text)

  # display rsponse as it is streamed
  print(chunk.text, end = '')

# render the finished response as formatted markdown
clear_output()
Markdown(buffer.getvalue())

The youngest author listed on the "Attention is All You Need" paper (which introduced the Transformer model) is **Aidan N. Gomez**.

While precise birthdates aren't always publicly available for researchers, based on publicly available information such as:

* **His academic trajectory:** He was a PhD student at the University of Oxford at the time of the paper's publication (2017). PhD students are generally younger than professors or established researchers.
* **His subsequent career stage:**  He co-founded Cohere, an AI company, after his PhD. This is typical for researchers who are earlier in their careers and looking to build upon their research.
* **General online presence and professional profiles:**  His profile generally suggests a more recent entry into the field compared to some of the other authors who are more established figures.

The other authors on the paper are more senior researchers with longer academic or industry careers, making Aidan N. Gomez the most likely youngest author at the time of publication.

Therefore, **Aidan N. Gomez** is widely considered to be the youngest author on the Transformer paper.